In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import warnings

from datetime import time, timedelta, datetime

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# Cleaned Data
- X Date
- X Unique Patient ID
- X Patient Tracker Order
- X Version of Patient Tracker
- X PCP or Specialtiy
- X Language
- Time: Time Arrived (Front Desk)
- Time: Checkpoint Recieves Clipboard (Checkpoint)
- Time: Student Doctor Receives Clipboard (Checkpoint)
- Time: Student Doctor ready to staff / placed in queue for physician (Checkpoint)
- Time: Physician Staffing Start (Checkpoint)
- Time: Physician Staffing End (Checkpoint)
- Time: Patient Checks Out (Front Desk)

In [2]:
patient_tracker_v1_end_date = "11.01.25"

In [3]:
#time_data_all = pd.DataFrame()

time_data_list = [] 

for patient_visit_tracker in sorted(os.listdir('./Data/SCU Patient Trackers/')):
    print(patient_visit_tracker)

    #############################################################################
    # Import data from patient visit tracker: checkpoint tracker table
    
    time_data = pd.read_excel(
        f"./Data/SCU Patient Trackers/{patient_visit_tracker}",
        sheet_name='Checkpoint Tracker (Checkpoint)',
        header=2
    )

    lang_data = pd.read_excel(
        f"./Data/SCU Patient Trackers/{patient_visit_tracker}",
        sheet_name='Intake Form Tracker (FrontDesk)',
        header=1
    )

    # Remove rows where BOTH Patient name AND DOB are empty/null
    time_data = time_data[~(time_data["Patient Name"].isna() & time_data['DOB'].isna())]

    # Remove rows where BOTH Patient name AND DOB are empty/null
    lang_data = lang_data[~(lang_data["Patient Name"].isna() & lang_data['DOB'].isna())]

    ############################################################################

    ############################################################################
    # Add language to time data
    
    lang_data["DOB"] = pd.to_datetime(
        lang_data["DOB"],
        errors='coerce'
    )

    time_data["DOB"] = pd.to_datetime(
        time_data["DOB"],
        errors='coerce'
    )

    #print(f"len of lang data: {len(lang_data)}")
    #print(f"len of time data pre merge: {len(time_data)}")

    #print(time_data)

    time_data = time_data.merge(
        lang_data[["DOB", "Patient Name", "Language"]], 
        on=["DOB", "Patient Name"], 
        how="left"
    )

    #print(f"len of time data post merge: {len(time_data)}")

    #print(time_data)
    #break
    
    ############################################################################
    
    ############################################################################
    # Adds the order in which the patient is listed in the tracker
    
    number_of_example_rows = (
        sum(
            ['Al Demo' in i for i in time_data['Patient Name'].dropna()]
        ) + 
        sum(
            ['Britney Spears ' in i for i in time_data['Patient Name'].dropna()]
        )
    )
    
    time_data = time_data[time_data['Patient Name'] != 'Al Demo']
    time_data = time_data[time_data['Patient Name'] != 'Britney Spears ']

    time_data["Patient Tracker Order"] = (
        time_data.index - number_of_example_rows
    )
    
    ############################################################################
    
    ############################################################################
    # Get the date from the file name and add it to each row
    match = re.search(r'\d{1,2}\.\d{1,2}\.\d{2,4}', patient_visit_tracker)
    if match:
        date_str = match.group()
        date = pd.to_datetime(date_str, format='%m.%d.%Y', errors='coerce')
        
        if pd.isna(date):  # Try 2-digit year format if 4-digit failed
            date = pd.to_datetime(date_str, format='%m.%d.%y')
    
        time_data["date"] = [ date ] * len(time_data)

        ###########################################################################
        # determine which version of the patient tracker is being used

        if date.date() <= datetime.strptime(patient_tracker_v1_end_date, "%m.%d.%y").date():
            time_data["Patient Tracker Version"] = "v1"
        else:
            time_data["Patient Tracker Version"] = "v2"

        ###########################################################################

    ############################################################################
    # give each patient a unique id that isn't name or dob to deidentify

    time_data["Unique ID"] = ""

    for i in time_data["Unique ID"].index:
        try:
            time_data.loc[i,"Unique ID"] = str(
                str(time_data.loc[i,"date"].date()) + str(time_data.loc[i,'DOB'])[2:4] + str(time_data.loc[i,"Patient Tracker Order"]) + str(time_data.loc[i,'Patient Name'])[-2::]
            )
        except Exception as e:
            pass
    ############################################################################
    
    # Append to list
    time_data_list.append(time_data)

# Concatenate all at once - pandas will handle column alignment automatically
time_data_all_raw = pd.concat(time_data_list, ignore_index=True)

Patient Visit Tracker (10.25.25).xlsx
Patient Visit Tracker (11.01.25).xlsx
Patient Visit Tracker (11.08.25).xlsx
Patient Visit Tracker (11.15.25).xlsx
Patient Visit Tracker (12.06.2025).xlsx
Patient Visit Tracker (12.13.2025).xlsx
Patient Visit Tracker - UPDATED (12.20.2025).xlsx
Patient Visit Tracker 10.04.2025.xlsx
Patient Visit Tracker 10.11.2025.xlsx
Patient Visit Tracker 10.18.25.xlsx


In [4]:
time_pattern = r'^\d{2}:\d{2}:\d{2}$'

# Create a mask for rows that DON'T match the pattern
mask = ~time_data_all_raw["PCP or Specialty"].apply(lambda x: bool(re.match(time_pattern, str(x))))

# Keep only rows where the mask is True
time_data_all_raw = time_data_all_raw[mask]
time_data_all_raw.reset_index(inplace=True)

In [5]:
time_data_all_clean = time_data_all_raw[[
    "date",
    "Unique ID",
    "Patient Tracker Order",
    "Patient Tracker Version",
    'PCP or Specialty',
    'Language'
]]

In [6]:
time_col_to_clean = [
    'Time: Time Arrived (Front Desk)',
    'Time: Checkpoint Recieves Clipboard (Checkpoint)',
    'Time: Student Doctor Receives Clipboard (Checkpoint)',
    'Time: Student Doctor ready to staff / placed in queue for physician \n(Checkpoint)',
    "Time: Physician Staffing Start (Checkpoint)",
    'Time: Physician Staffing End (Checkpoint)',
    "Time: Patient Checks Out (Front Desk)"
]

# Make an explicit copy if time_data_all_clean is a slice
time_data_all_clean = time_data_all_clean.copy()

for col_to_clean in time_col_to_clean:
    # Clean times that were entered wrong in excel
    for i in range(len(time_data_all_raw[col_to_clean])):
        if type(time_data_all_raw.loc[i, col_to_clean]) is str:
            # Found issue 1
            # Example 9:40AM should be 09:40:00
            time_data_all_raw.loc[i, col_to_clean] = re.sub(
                r'(:\d{2})[a-zA-Z]{2}',  # pattern: lowercase letters
                r'\1:00',  # replacement: empty string (removes the letters)
                time_data_all_raw.loc[i, col_to_clean]  # string to operate on
            )

    
    # Convert to datetime
    time_data_all_clean[col_to_clean] = pd.to_datetime(
        time_data_all_raw['date'].astype(str) + ' ' + time_data_all_raw[col_to_clean].astype(str),
        format='%Y-%m-%d %H:%M:%S',
        errors='coerce'
    )
    
    # Converts non-military to military time
    # Use boolean indexing instead of iterating
    mask = (
        pd.notna(time_data_all_clean[col_to_clean]) &
        (time_data_all_clean[col_to_clean].dt.time < time(7, 30))
    )
    time_data_all_clean.loc[mask, col_to_clean] += timedelta(hours=12)

In [7]:
time_data_all_clean['Canceled'] = time_data_all_clean[time_col_to_clean].isna().all(axis=1)

#Fixing colum order so canceled appears before time
cols = time_data_all_clean.columns.tolist()
cols = cols[:6] + cols[-1:] + cols[6:-1]

time_data_all_clean = time_data_all_clean[cols]

In [8]:
time_data_all_clean

,date,Unique ID,Patient Tracker Order,Patient Tracker Version,PCP or Specialty,Language,Canceled,Time: Time Arrived (Front Desk),Time: Checkpoint Recieves Clipboard (Checkpoint),Time: Student Doctor Receives Clipboard (Checkpoint),Time: Student Doctor ready to staff / placed in queue for physician \n(Checkpoint),Time: Physician Staffing Start (Checkpoint),Time: Physician Staffing End (Checkpoint),Time: Patient Checks Out (Front Desk)
0,2025-10-25,2025-10-25760B.,0,v1,PCP,English,True,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2025-10-25,2025-10-25921H.,1,v1,PCP,English,False,2025-10-25 08:30:00,2025-10-25 08:56:00,NaT,2025-10-25 09:19:00,2025-10-25 09:45:00,NaT,2025-10-25 11:15:00
2,2025-10-25,2025-10-25642S.,2,v1,PCP,Ukranian,False,2025-10-25 09:00:00,2025-10-25 09:40:00,NaT,2025-10-25 10:06:00,2025-10-25 10:30:00,NaT,2025-10-25 11:30:00
3,2025-10-25,2025-10-25543T.,3,v1,PCP,English,False,2025-10-25 10:05:00,2025-10-25 10:48:00,NaT,2025-10-25 11:40:00,2025-10-25 12:17:00,NaT,2025-10-25 14:12:00
4,2025-10-25,2025-10-25024A.,4,v1,PCP,English,False,2025-10-25 10:10:00,2025-10-25 10:40:00,NaT,2025-10-25 11:05:00,2025-10-25 11:20:00,NaT,2025-10-25 12:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2025-10-18,2025-10-187216an,16,v1,PCP,English,False,2025-10-18 10:11:00,2025-10-18 10:21:00,NaT,2025-10-18 10:55:00,2025-10-18 11:30:00,NaT,2025-10-18 13:32:00
146,2025-10-18,2025-10-185917an,17,v1,PCP,Spanish,False,2025-10-18 10:15:00,2025-10-18 10:43:00,NaT,2025-10-18 11:40:00,2025-10-18 12:00:00,NaT,2025-10-18 14:11:00
147,2025-10-18,2025-10-186518na,18,v1,PCP,Russian,False,2025-10-18 10:10:00,2025-10-18 10:21:00,NaT,2025-10-18 10:56:00,2025-10-18 11:00:00,NaT,NaT
148,2025-10-18,2025-10-189519an,19,v1,PCP,English,False,2025-10-18 10:25:00,2025-10-18 10:39:00,NaT,2025-10-18 11:15:00,2025-10-18 11:30:00,NaT,2025-10-18 12:00:00


In [9]:
time_data_all_clean.to_csv("patient_visit_tracker.csv")

# Future cleaning 
- patient visit tracker 10.11.2025
   - has a bunch of random patients at the bottom of tracker not sure why pt 10 - 19

# Verifying Cleaning

## Future tests

## Tests

In [10]:
# Check if all rows have at least three non na values
pass_check = True

rows_that_fail_this_check = []

for i, row in time_data_all_clean.iterrows():
    if row[time_col_to_clean].notna().sum() < 3 and not row["Canceled"]:
        pass_check = False
        rows_that_fail_this_check.append(row) 
        print(row)        

print(
    f"All rows have atleast three non na values: {pass_check}\nPercent that fail this test: {len(rows_that_fail_this_check) / len(time_data_all_clean) * 100}%"
)

date                                                                                  2025-10-04 00:00:00
Unique ID                                                                                 2025-10-04965ve
Patient Tracker Order                                                                                   5
Patient Tracker Version                                                                                v1
PCP or Specialty                                                                                     Derm
Language                                                                                          English
Canceled                                                                                            False
Time: Time Arrived (Front Desk)                                                       2025-10-04 09:09:00
Time: Checkpoint Recieves Clipboard (Checkpoint)                                                      NaT
Time: Student Doctor Receives Clipboard (Check

In [11]:
# Check if all rows that canceled have a PCP and language
pass_check = True

rows_that_fail_this_check = []

for i, row in time_data_all_clean.iterrows():
    if pd.isna(row["Language"]) and pd.isna(row["PCP or Specialty"]):
        pass_check = False
        rows_that_fail_this_check.append(row) 
        print(row)        

print(
    f"All canceled rows have a Language and PCP: {pass_check}\nPercent that fail this test: {len(rows_that_fail_this_check) / len(time_data_all_clean) * 100}%"
)

date                                                                                  2025-10-18 00:00:00
Unique ID                                                                                  2025-10-18T0on
Patient Tracker Order                                                                                   0
Patient Tracker Version                                                                                v1
PCP or Specialty                                                                                      NaN
Language                                                                                              NaN
Canceled                                                                                             True
Time: Time Arrived (Front Desk)                                                                       NaT
Time: Checkpoint Recieves Clipboard (Checkpoint)                                                      NaT
Time: Student Doctor Receives Clipboard (Check

In [12]:
# Check if all rows that canceled have a PCP and language
pass_check = True

rows_that_fail_this_check = []

for i, row in time_data_all_clean.iterrows():
    if any(char.isdigit() for char in str(row["PCP or Specialty"])):
        pass_check = False
        rows_that_fail_this_check.append(row) 
        print(row)        

print(
    f"All canceled rows have a Language and PCP: {pass_check}\nPercent that fail this test: {len(rows_that_fail_this_check) / len(time_data_all_clean) * 100}%"
)

date                                                                                  2025-11-15 00:00:00
Unique ID                                                                                 2025-11-15700is
Patient Tracker Order                                                                                   0
Patient Tracker Version                                                                                v2
PCP or Specialty                                                                                       G2
Language                                                                                          English
Canceled                                                                                            False
Time: Time Arrived (Front Desk)                                                       2025-11-15 08:33:00
Time: Checkpoint Recieves Clipboard (Checkpoint)                                                      NaT
Time: Student Doctor Receives Clipboard (Check

In [13]:
# Check if all time is less than six hours
pass_check = True

for i, row in time_data_all_clean.iterrows():
    if (row["Time: Patient Checks Out (Front Desk)"] - row["Time: Time Arrived (Front Desk)"]).total_seconds() > 6*60*60:
        pass_check = False
        print(row)        

print(
    f"All visit times are less than six hours: {pass_check}"
)

All visit times are less than six hours: True


In [14]:
# Check a that all total time is positive
pass_check = True

for i, row in time_data_all_clean.iterrows():
    if (row["Time: Patient Checks Out (Front Desk)"] - row["Time: Time Arrived (Front Desk)"]).total_seconds() < 0:
        pass_check = False
        print(row)        

print(
    f"All visit times are positive: {pass_check}"
)

All visit times are positive: True


In [15]:
# Test if all the differences are positive
pass_pos_val_check = [ True ] * ( len(time_col_to_clean) - 1 )

for test_in in range(len(time_col_to_clean)):
    if test_in > 0:

        print(f"\nTEST: All visit {time_col_to_clean[test_in - 1]} to {time_col_to_clean[test_in]} are positive\n")
    
        for i, row in time_data_all_clean.iterrows():
            if (row[time_col_to_clean[test_in]] - row[time_col_to_clean[test_in - 1]]).total_seconds() < 0:
                pass_pos_val_check[test_in - 1] = False
                print(row)   

        print(f"\nTEST result: {pass_pos_val_check[test_in - 1]}")

print(f"\nThe percent of differences that pass this check: {sum(pass_pos_val_check) / len(pass_pos_val_check) * 100}%")


TEST: All visit Time: Time Arrived (Front Desk) to Time: Checkpoint Recieves Clipboard (Checkpoint) are positive

date                                                                                  2025-12-06 00:00:00
Unique ID                                                                                2025-12-067311iz
Patient Tracker Order                                                                                  11
Patient Tracker Version                                                                                v2
PCP or Specialty                                                                                      PCP
Language                                                                               Spanish, Castilian
Canceled                                                                                            False
Time: Time Arrived (Front Desk)                                                       2025-12-06 10:50:00
Time: Checkpoint Recieves Clipboard (

## Distributions

In [16]:
for i in time_col_to_clean[1::]:
    print(f"\n{i}:")
    print(((time_data_all_clean[i] - time_data_all_clean['Time: Time Arrived (Front Desk)']).dropna().dt.total_seconds() / 60).describe())


Time: Checkpoint Recieves Clipboard (Checkpoint):
count    91.000000
mean     19.945055
std      11.020549
min     -37.000000
25%      14.000000
50%      19.000000
75%      26.000000
max      50.000000
dtype: float64

Time: Student Doctor Receives Clipboard (Checkpoint):
count     53.000000
mean      28.773585
std       24.905700
min      -32.000000
25%       19.000000
50%       25.000000
75%       30.000000
max      152.000000
dtype: float64

Time: Student Doctor ready to staff / placed in queue for physician 
(Checkpoint):
count     83.000000
mean      63.927711
std       30.067316
min      -87.000000
25%       49.000000
50%       62.000000
75%       78.000000
max      173.000000
dtype: float64

Time: Physician Staffing Start (Checkpoint):
count     83.000000
mean      86.120482
std       39.805747
min      -87.000000
25%       56.500000
50%       86.000000
75%      109.500000
max      245.000000
dtype: float64

Time: Physician Staffing End (Checkpoint):
count     41.00000
mean     